In [1]:
import tensorflow as tf
from tensorflow import keras
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization

2023-12-22 13:07:27.715305: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-22 13:07:27.718701: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-12-22 13:07:27.718716: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
# !pip install tflearn 

In [4]:
# Get Data
import tflearn.datasets.oxflower17 as oxflower17
x, y = oxflower17.load_data(one_hot=True)

Instructions for updating:
non-resource variables are not supported in the long term


100.0% 60276736 / 60270631


Succesfully downloaded 17flowers.tgz 60270631 bytes.
File Extracted
Starting to parse images...
Parsing Done!


In [5]:
x

array([[[[0.00784314, 0.01568628, 0.01176471],
         [0.00784314, 0.01568628, 0.01176471],
         [0.00784314, 0.01568628, 0.01176471],
         ...,
         [0.00392157, 0.01176471, 0.00784314],
         [0.00392157, 0.01176471, 0.00784314],
         [0.00392157, 0.01176471, 0.00784314]],

        [[0.00784314, 0.01568628, 0.01176471],
         [0.00784314, 0.01568628, 0.01176471],
         [0.00784314, 0.01568628, 0.01176471],
         ...,
         [0.00392157, 0.01176471, 0.00784314],
         [0.00392157, 0.01176471, 0.00784314],
         [0.00392157, 0.01176471, 0.00784314]],

        [[0.00784314, 0.01568628, 0.01176471],
         [0.00784314, 0.01568628, 0.01176471],
         [0.00784314, 0.01568628, 0.01176471],
         ...,
         [0.00784314, 0.01568628, 0.01176471],
         [0.00784314, 0.01568628, 0.01176471],
         [0.00784314, 0.01568628, 0.01176471]],

        ...,

        [[0.00784314, 0.01568628, 0.01176471],
         [0.00784314, 0.01568628, 0.01176471]

In [6]:
y

array([[0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.]])

In [7]:
x.shape

(1360, 224, 224, 3)

In [8]:
y.shape

(1360, 17)

In [9]:
# Create a sequential model
model = Sequential()

# 1st Convolutional Layer
model.add(Conv2D(filters=96, input_shape=(224,224,3), kernel_size=(11,11), strides=(4,4), padding='valid'))
model.add(Activation('relu'))

# Pooling 
model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid'))
# Batch Normalisation before passing it to the next layer
model.add(BatchNormalization())

# 2nd Convolutional Layer
model.add(Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), padding='same'))
model.add(Activation('relu'))

# Pooling
model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid'))
# Batch Normalisation
model.add(BatchNormalization())



# 3rd Convolutional Layer
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
# Batch Normalisation
model.add(BatchNormalization())

# 4th Convolutional Layer
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
# Batch Normalisation
model.add(BatchNormalization())


# 5th Convolutional Layer
model.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='valid'))
model.add(Activation('relu'))


# Pooling
model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid'))
# Batch Normalisation
model.add(BatchNormalization())


# Passing it to a dense layer
model.add(Flatten())

# 1st Dense Layer
model.add(Dense(4096, input_shape=(224*224*3,)))
model.add(Activation('relu'))
# Add Dropout to prevent overfitting
model.add(Dropout(0.4))
# Batch Normalisation
model.add(BatchNormalization())

# 2nd Dense Layer
model.add(Dense(4096))
model.add(Activation('relu'))
# Add Dropout
model.add(Dropout(0.4))
# Batch Normalisation
model.add(BatchNormalization())

# Output Layer
model.add(Dense(17))
model.add(Activation('softmax'))

model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 54, 54, 96)        34944     
                                                                 
 activation (Activation)     (None, 54, 54, 96)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 26, 26, 96)       0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 26, 26, 96)       384       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 26, 26, 256)       614656    
                                                  

In [11]:
# !pip install tensorflow-addons==0.16.1

In [12]:
# Compile 
opt = tf.optimizers.Adam(learning_rate = 0.01)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [13]:
# Train
model.fit(x, y, batch_size=64, epochs=5, verbose=1,validation_split=0.2, shuffle=True)

Train on 1088 samples, validate on 272 samples


2023-12-22 13:11:28.119332: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-12-22 13:11:28.119353: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-12-22 13:11:28.119370: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ZSCHN01LP0253L): /proc/driver/nvidia/version does not exist
2023-12-22 13:11:28.120491: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-22 13:11:28.156496: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:3

Epoch 1/5
1088/1088 [==============================] - ETA: 0s - loss: 9.1978 - acc: 0.1351

/home/pradeep.k@zucisystems.com/workspace_working_Repo/ML/envname/lib/python3.8/site-packages/keras/engine/training_v1.py:2045: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


1088/1088 [==============================] - 23s 21ms/sample - loss: 9.1978 - acc: 0.1351 - val_loss: 200815.8585 - val_acc: 0.0478
Epoch 2/5
1088/1088 [==============================] - 21s 19ms/sample - loss: 6.2788 - acc: 0.2105 - val_loss: 8530.4275 - val_acc: 0.0846
Epoch 3/5
1088/1088 [==============================] - 23s 21ms/sample - loss: 5.2284 - acc: 0.2472 - val_loss: 1257.9321 - val_acc: 0.0882
Epoch 4/5
1088/1088 [==============================] - 23s 21ms/sample - loss: 4.7739 - acc: 0.2776 - val_loss: 596.3220 - val_acc: 0.0404
Epoch 5/5
1088/1088 [==============================] - 23s 21ms/sample - loss: 3.6682 - acc: 0.3272 - val_loss: 113.4686 - val_acc: 0.0919
